In [18]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy 



In [19]:
## Loading all the models 

model = load_model('model.h5')

with open('label_enc_gender.pkl','rb') as file:
    label_enc_gender = pickle.load(file)
    
with open('one_hot_geo.pkl','rb') as file:
    one_hot_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
  scaler = pickle.load(file)

In [20]:
input_data = {
    'CreditScore': 450,        # Lower credit score
    'Geography': 'Germany',    # Germany may have higher churn rates in some models
    'Gender': 'Female',        # Some models show gender bias depending on the dataset
    'Age': 65,                 # Older age can be a factor in churn
    'Tenure': 1,               # Shorter tenure indicates newer customer
    'Balance': 150000,         # Higher balance but inactive member
    'NumOfProducts': 1,        # Fewer products typically indicate lower engagement
    'HasCrCard': 0,            # Not having a credit card might indicate lower loyalty
    'IsActiveMember': 0,       # Inactive members are more likely to churn
    'EstimatedSalary': 80000   # Salary might be less impactful, but a higher salary can be associated with higher balance
}


In [21]:
geo_encoded = one_hot_geo.transform([[input_data['Geography']]]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded,columns = one_hot_geo.get_feature_names_out(['Geography']))

c:\ProgramData\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [22]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [23]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,450,Germany,Female,65,1,150000,1,0,0,80000


In [25]:
input_df.reset_index(drop=True)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,450,Germany,Female,65,1,150000,1,0,0,80000


In [ ]:
# input_data = pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)

In [ ]:
input_data

{'CreditScore': 450,
 'Geography': 'Germany',
 'Gender': 'Female',
 'Age': 65,
 'Tenure': 1,
 'Balance': 150000,
 'NumOfProducts': 1,
 'HasCrCard': 0,
 'IsActiveMember': 0,
 'EstimatedSalary': 80000}

In [ ]:
input_df['Gender'] = label_enc_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,450,Germany,0,65,1,150000,1,0,0,80000


In [ ]:
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [ ]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,450,0,65,1,150000,1,0,0,80000,0.0,1.0,0.0


In [ ]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-2.06946052, -1.10665898,  2.48614504, -1.38393756,  1.17096021,
        -0.91014926, -1.5482337 , -1.02429504, -0.3448289 , -0.996257  ,
         1.70645259, -0.57157662]])

In [ ]:
prediction = model.predict(input_scaled)

1/1 [==============================] - 4s 4s/step


In [ ]:
prediction[0]

array([0.86015636], dtype=float32)

In [ ]:
if(prediction[0][0] > 0.5) :
  print("The Customer is likely to Churn")

else :
  print("The costomer is not likely to churn")


The Customer is likely to Churn
